# Homework
This notebook guides the user through implementing a CNN using PyTorch. The architecture and data set should match those present in the TensorFlow notebook.

## Steps:
- Select data-set
- Define Architecture
- Instantiate CNN
- Train and Test
- Output Convolution layers and results
- thoroughly display test results and model performance.

# TODO
Transpose (this PyTorch tutorial)[https://medium.com/@myringoleMLGOD/simple-convolutional-neural-network-cnn-for-dummies-in-pytorch-a-step-by-step-guide-6f4109f6df80] into this notebook.


Update the tutorial to use (this bird classification dataset)[https://www.kaggle.com/datasets/rahmasleam/bird-speciees-dataset/data]

Filters (Kernels):

    Filters are small matrices that slide over the input image and perform element-wise multiplications followed by summation. Each filter is designed to detect a specific feature in the input image.
    For example, a filter might detect horizontal edges, vertical edges, or more complex textures.
    The output of applying a filter to the input image is called a feature map or activation map. If you have multiple filters, you will get multiple feature maps.

Stride:

    Stride is the step size with which the filter moves across the input image.
    A stride of 1 means the filter moves one pixel at a time, both horizontally and vertically.
    A larger stride reduces the size of the feature map because the filter skips more pixels. For instance, a stride of 2 means the filter moves two pixels at a time, effectively down-sampling the feature map.

Padding:

    Padding involves adding extra pixels around the input image’s border. These extra pixels are typically set to zero (zero-padding).
    Padding ensures that the filter fits properly over the image, especially at the edges. Without padding, the feature map’s size reduces after each convolution operation.
    For example, if you have a 5x5 input image and a 3x3 filter with no padding, the resulting feature map will be 3x3. With padding of 1, the feature map remains the same size as the input.

Feature Map:

    A feature map is the output of a convolutional layer after applying filters to the input image.
    Each feature map corresponds to a different filter and captures different features from the input.
    Stacking multiple feature maps together forms a multi-channel output that serves as the input for the next layer.

Pooling Layers

Pooling layers reduce the spatial dimensions of the feature maps, which helps in making the network computationally efficient and reducing overfitting. There are two main types of pooling:

    Max Pooling:

    Max pooling takes the maximum value from each patch of the feature map.
    For example, in a 2x2 max pooling operation, the maximum value from each 2x2 block of the feature map is taken to create a new, smaller feature map.
    This operation reduces the size of the feature map by half, both horizontally and vertically, but retains the most prominent features.

    Average Pooling:

    Average pooling takes the average value from each patch of the feature map.
    Similar to max pooling, but instead of the maximum value, it takes the average value from each block.
    This can be useful in different contexts, though max pooling is more common in practice.

In [ ]:
# Imports
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
# Define CNN Module

class CNN(nn.Module):
    def __init__(self, in_channels, num_classes=10):
        """
        Define the layers of the convolutional neural network.

        Parameters:
            in_channels: int
                The number of channels in the input image. For MNIST, this is 1 (grayscale images).
            num_classes: int
                The number of classes we want to predict, in our case 10 (digits 0 to 9).
        """
        super(CNN, self).__init__() # Add an explanation of what super() does for us here and Link to documentation.

        # First convolutional layer: 1 input channel, 8 output channels, 3x3 kernel, stride 1, padding 1
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=8, kernel_size=3, stride=1, padding=1)
        # Max pooling layer: 2x2 window, stride 2
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Second convolutional layer: 8 input channels, 16 output channels, 3x3 kernel, stride 1, padding 1
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1)
        # Fully connected layer: 16*7*7 input features (after two 2x2 poolings), 10 output features (num_classes)
        self.fc1 = nn.Linear(16 * 7 * 7, num_classes)

    def forward(self, x):
        """
        Define the forward pass of the neural network.
        TODO: Add a description of what a forward pass is.

        Parameters:
            x: torch.Tensor
                The input tensor.

        Returns:
            torch.Tensor
                The output tensor after passing through the network.
        """
        x = F.relu(self.conv1(x))  # Apply first convolution and ReLU activation
        x = self.pool(x)           # Apply max pooling
        x = F.relu(self.conv2(x))  # Apply second convolution and ReLU activation
        x = self.pool(x)           # Apply max pooling
        x = x.reshape(x.shape[0], -1)  # Flatten the tensor
        x = self.fc1(x)            # Apply fully connected layer
        return x 

In [ ]:
# Use GPU if Available
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Define Model-Specific Variables
input_size = 784  # 28x28 pixels (not directly used in CNN)
num_classes = 10  # digits 0-9
learning_rate = 0.001
batch_size = 64
num_epochs = 10  # Reduced for demonstration purposes
criterion = nn.CrossEntropyLoss()

In [ ]:
# Initalize Model and Optimizer
model = CNN(in_channels=1, num_classes=num_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Fetch and Split Data
train_dataset = datasets.MNIST(root="dataset/", download=True, train=True, transform=transforms.ToTensor())
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root="dataset/", download=True, train=False, transform=transforms.ToTensor())
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# Train the Model
for epoch in range(num_epochs):
    print(f"Epoch [{epoch + 1}/{num_epochs}]")
    for batch_index, (data, targets) in enumerate(tqdm(train_loader)):
        # Move data and targets to the device (GPU/CPU)
        data = data.to(device)
        targets = targets.to(device)

        # Forward pass: compute the model output
        scores = model(data)
        loss = criterion(scores, targets)

        # Backward pass: compute the gradients
        optimizer.zero_grad()
        loss.backward()

        # Optimization step: update the model parameters
        optimizer.step()

In [ ]:
# Evaluate Model
# TODO: Expand this section to include ROC, Confusion Matrix, ect...
def check_accuracy(loader, model):
    """
    Checks the accuracy of the model on the given dataset loader.

    Parameters:
        loader: DataLoader
            The DataLoader for the dataset to check accuracy on.
        model: nn.Module
            The neural network model.
    """
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0
    model.eval()  # Set the model to evaluation mode

    with torch.no_grad():  # Disable gradient calculation
        for x, y in loader: # update to have a descriptive name for x nd y
            x = x.to(device)
            y = y.to(device)

            # Forward pass: compute the model output
            scores = model(x)
            _, predictions = scores.max(1)  # Get the index of the max log-probability
            num_correct += (predictions == y).sum()  # Count correct predictions
            num_samples += predictions.size(0)  # Count total samples

        # Calculate accuracy
        accuracy = float(num_correct) / float(num_samples) * 100
        print(f"Got {num_correct}/{num_samples} with accuracy {accuracy:.2f}%")
    
    model.train()  # Set the model back to training mode

# Final accuracy check on training and test sets
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)
